# Building AI-Powered Flood Prediction System with h2oGPTe and NVIDIA NIM

[![Deploy on NVIDIA](https://img.shields.io/badge/Deploy%20on-NVIDIA%20AI%20Blueprints-76B900?logo=nvidia&logoColor=white)](https://build.nvidia.com)
[![H2O.ai](https://img.shields.io/badge/Powered%20by-H2O.ai-FFD500)](https://h2o.ai)

---

## 🌊 Overview

This blueprint demonstrates an **AI-powered flood prediction and disaster response system** that combines:

- **h2oGPTe Agent-to-Agent (A2A)**: Advanced AutoML capabilities with Driverless AI for model training and feature engineering
- **NVIDIA NIM**: State-of-the-art inference with `nvidia/llama-3.3-nemotron-super-49b-v1.5` and other NVIDIA models
- **NVIDIA NAT Pipeline**: React Agent workflows for multi-agent orchestration
- **FastMCP Server**: 20+ specialized tools across 5 intelligent agents
- **Real-time Data Integration**: USGS Water Services, NOAA Forecasts, and Weather APIs

### 🎯 Use Case: AI for Good - Disaster Response

This system provides:
- **Real-time flood monitoring** with live data from watersheds and monitoring stations
- **AI-powered risk assessment** using advanced machine learning models
- **Emergency response coordination** with automated alerts and evacuation planning
- **Predictive analytics** for flood forecasting 24-72 hours ahead
- **AutoML model training** for continuous improvement of prediction accuracy

### 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                      Flood Prediction System                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐         │
│  │   h2oGPTe    │  │  NVIDIA NIM  │  │  FastMCP     │         │
│  │  (A2A Mode)  │  │  (Nemotron)  │  │   Server     │         │
│  └──────┬───────┘  └──────┬───────┘  └──────┬───────┘         │
│         │                 │                 │                   │
│         └────────┬────────┴────────┬────────┘                   │
│                  │                 │                            │
│         ┌────────▼─────────────────▼────────┐                  │
│         │    NVIDIA NAT Agent Pipeline      │                  │
│         │      (React Agent Workflow)       │                  │
│         └────────┬──────────────────────────┘                  │
│                  │                                              │
│    ┌─────────────┴─────────────┐                               │
│    │    5 Specialized Agents   │                               │
│    ├───────────────────────────┤                               │
│    │  1. Data Collector        │ ◄── USGS Water Data           │
│    │  2. Risk Analyzer         │ ◄── NOAA Flood Alerts         │
│    │  3. Emergency Responder   │ ◄── Weather APIs              │
│    │  4. AI Predictor          │                               │
│    │  5. H2OGPTE ML Agent      │                               │
│    └───────────────────────────┘                               │
│                                                                  │
│  Output: Real-time Monitoring, Alerts, Predictions, ML Models  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔑 Key Technologies

1. **h2oGPTe**: Enterprise AI platform with agent mode for AutoML and advanced analytics
2. **NVIDIA NIM**: Optimized inference microservices for AI models
3. **NVIDIA NAT**: Agent orchestration framework with React-based workflows
4. **FastMCP**: Model Context Protocol server for tool integration
5. **FastAPI**: High-performance API server for real-time operations

---

## 📋 What You'll Learn

- Setting up multi-agent AI systems for disaster response
- Integrating h2oGPTe for AutoML and model training
- Using NVIDIA NIM for high-performance inference
- Building NAT agent workflows with React patterns
- Implementing FastMCP servers with custom tools
- Real-time data integration from government APIs
- Coordinating multiple AI agents for complex tasks

Let's get started! 🚀

## 🚀 Helm Deployment

Before running this notebook, deploy the backend services using the Helm chart described in `./deployment/helm/README.md`.

## 🔐 Environment Configuration

Ensure the required API credentials and service endpoints are available in the environment before proceeding.

In [1]:
import json
import os
import time

import requests
from openai import OpenAI


def preview_key(value: str) -> str:
    if not value:
        return "Not set"
    if len(value) <= 14:
        return value
    dots = max(1, len(value) - 14)
    return f"{value[:10]}{'.' * dots}{value[-4:]}"


NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
print(f"NVIDIA_API_KEY: {preview_key(NVIDIA_API_KEY)}")

H2OGPTE_URL = os.getenv("APP_H2OGPTE_URL")
print(f"APP_H2OGPTE_URL: {H2OGPTE_URL or 'Not set'}")

H2OGPTE_API_KEY = os.getenv("APP_H2OGPTE_API_KEY")
print(f"APP_H2OGPTE_API_KEY: {preview_key(H2OGPTE_API_KEY)}")

if H2OGPTE_URL and H2OGPTE_API_KEY:
    print("✅ h2oGPTe credentials found")
    print(f"🔗 URL: {H2OGPTE_URL}")
    H2OGPTE_AVAILABLE = True
else:
    print("⚠️  h2oGPTe credentials not set")
    print("This section will be skipped. To enable:")
    print("  export APP_H2OGPTE_URL='<your-url>'")
    print("  export APP_H2OGPTE_API_KEY='<your-key>'")
    H2OGPTE_AVAILABLE = False

api_server_base_url = os.getenv("API_SERVER_URL")
if api_server_base_url:
    print(f"✅ API_SERVER_URL detected: {api_server_base_url}")
else:
    print("⚠️  API_SERVER_URL not set. API interactions will be skipped.")
api_server_base_url


NVIDIA_API_KEY: nvapi-LRQz........................................................5JXH
APP_H2OGPTE_URL: https://h2ogpte.cloud-dev.h2o.dev
APP_H2OGPTE_API_KEY: sk-ktl41qb.....................................tiqB
✅ h2oGPTe credentials found
🔗 URL: https://h2ogpte.cloud-dev.h2o.dev
✅ API_SERVER_URL detected: http://flood-pred-web


'http://flood-pred-web'

In [3]:
# Services are managed via the Helm deployment. Display key endpoints for convenience.
print("🔧 Helm-managed services are expected to be running.\n")

if not api_server_base_url:
    print("⚠️  API_SERVER_URL not set. Export API_SERVER_URL before continuing.")
else:
    base_url = api_server_base_url.rstrip('/')
    print("📍 Service Endpoints:")
    print(f"   - FastAPI Server: {base_url}")
    print(f"   - API Docs: {base_url}/docs")
    print(f"   - Agents API: {base_url}/api/agents")
    print("\nUse the next cell to verify service health.")

🔧 Helm-managed services are expected to be running.

📍 Service Endpoints:
   - FastAPI Server: http://flood-pred-web
   - API Docs: http://flood-pred-web/docs
   - Agents API: http://flood-pred-web/api/agents

Use the next cell to verify service health.


## ✅ Verify Services

Let's check that all services are running correctly:

In [4]:
import requests

print("🔍 Checking service health...\n")

if not api_server_base_url:
    print("⚠️  API_SERVER_URL not set. Skipping health checks.")
    fastapi_ok = False
else:
    base_url = api_server_base_url.rstrip('/')

    def check_service(path, name, headers=None):
        url = f"{base_url}/{path.lstrip('/')}"
        try:
            response = requests.get(url, headers=headers, timeout=10)
        except requests.exceptions.RequestException as exc:
            print(f"❌ {name} is not responding: {exc}")
            return False

        if response.status_code == 200:
            print(f"✅ {name} is running")
            return True
        if response.status_code == 401:
            print(f"⚠️  {name} requires authentication (401)")
            return True

        print(f"⚠️  {name} responded with status {response.status_code}")
        return False

    fastapi_ok = check_service('api/dashboard', 'FastAPI Dashboard')
    agents_ok = check_service('api/agents', 'Agents API', headers={'Authorization': 'Bearer local-token'})
    watersheds_ok = check_service('api/watersheds', 'Watersheds API')

    print("\n" + "=" * 50)
    if fastapi_ok and agents_ok and watersheds_ok:
        print("✅ All services are responding!")
    else:
        print("⚠️  One or more endpoints did not respond as expected.")
        print("Review Helm deployment status or API logs if needed.")
    print("=" * 50)

🔍 Checking service health...

✅ FastAPI Dashboard is running
✅ Agents API is running
✅ Watersheds API is running

✅ All services are responding!


---

# Section 2: NVIDIA NIM Integration

## 🚀 NVIDIA NIM - Optimized Inference Microservices

NVIDIA NIM provides high-performance inference for state-of-the-art language models. Our flood prediction system uses several NVIDIA models:

### Available Models

1. **nvidia/llama-3.3-nemotron-super-49b-v1.5** (Default)
   - Latest Nemotron model optimized for instruction following
   - Excellent for agent workflows and tool calling
   - 49B parameters with superior efficiency

2. **meta/llama-3.1-70b-instruct**
   - Strong general-purpose reasoning
   - Great for complex analysis tasks

3. **nvidia/llama-3.1-nemotron-70b-instruct**
   - NVIDIA-optimized Llama variant
   - Enhanced for enterprise use cases

4. **meta/llama-3.1-405b-instruct**
   - Used as LLM-as-Judge for evaluation
   - Highest reasoning capability

### Let's test NVIDIA NIM integration:

In [ ]:
from openai import OpenAI
import os

# Initialize NVIDIA NIM client
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv('NVIDIA_API_KEY')
)

# Test with Nemotron Super 49B
model = "nvidia/llama-3.3-nemotron-super-49b-v1.5"

local_nim_base_url: str = os.environ.get("NIM_LLM_BASE_URL", "")
local_nim_default_model: str = os.environ.get("NIM_LLM_DEFAULT_MODEL", "nvidia/llama-3-3-nemotron-super-49b-v1-5")

if local_nim_base_url:
    print(f"Using local NIM base URL: {local_nim_base_url}")
    client = OpenAI(
        base_url=local_nim_base_url,
        api_key="dummy",
    )

print(f"🤖 Testing NVIDIA NIM with {model}\n")

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are an expert flood prediction assistant."},
        {"role": "user", "content": "What are the key factors that indicate an increased risk of flooding in a river basin?"}
    ],
    temperature=0.7,
    max_tokens=500
)

print("📝 Response:")
print(response.choices[0].message.content)
print(f"\n📊 Tokens used: {response.usage.total_tokens}")

🤖 Testing NVIDIA NIM with nvidia/llama-3.3-nemotron-super-49b-v1.5

📝 Response:
<think>
Okay, so I need to figure out the key factors that indicate an increased risk of flooding in a river basin. Let me start by recalling what I know about floods and river basins. A river basin is the area of land that drains water into a particular river and its tributaries. Flooding happens when the river can't hold the amount of water it's receiving, leading to overflow.

First, I think precipitation is a major factor. If there's a lot of rain, especially heavy and prolonged rainfall, that can cause the river levels to rise quickly. But maybe it's not just the amount of rain, but also the intensity and duration. Like, a storm that dumps a lot of rain in a short time might be worse than a steady drizzle over days.

Then there's the terrain. If the land is steep, water runoff can be faster, leading to quicker rises in river levels. Flat areas might allow water to spread out more, but if the soil is sa

### Streaming Response Example

NVIDIA NIM supports streaming for real-time responses:

In [7]:
print("🌊 Streaming response about flood prediction...\n")

stream = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a concise flood prediction expert."},
        {"role": "user", "content": "Explain how AI can improve flood forecasting accuracy."}
    ],
    temperature=0.7,
    max_tokens=300,
    stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n\n✅ Streaming complete!")

🌊 Streaming response about flood prediction...

<think>
Okay, so I need to explain how AI can improve flood forecasting accuracy. Let me start by recalling what I know about flood forecasting and AI. Flood forecasting involves predicting the occurrence and extent of flooding, usually based on factors like rainfall, river levels, soil moisture, and weather patterns. Traditional methods might use hydrological models that are based on physical equations and historical data. But AI, especially machine learning, can offer some advantages here.

First, AI can handle large datasets. Flood prediction requires processing vast amounts of data from various sources like satellite imagery, weather stations, river gauges, and maybe even social media. Traditional models might struggle with integrating all these data sources efficiently, but AI can process and analyze them quickly. For example, machine learning algorithms can detect patterns in historical data that might not be obvious to traditional 

### Model Comparison

Let's compare responses from different NVIDIA models:

In [8]:
models_to_test = [
    "nvidia/llama-3.3-nemotron-super-49b-v1.5",
    "meta/llama-3.1-70b-instruct"
]

question = "Given streamflow of 2500 CFS and rising 200 CFS/hour, should we issue a flood alert?"

print(f"📊 Comparing model responses for:\n'{question}'\n")
print("="*80)

for model_name in models_to_test:
    print(f"\n🤖 Model: {model_name.split('/')[-1]}\n")
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a flood emergency expert. Be concise."},
                {"role": "user", "content": question}
            ],
            temperature=0.3,
            max_tokens=200
        )
        
        print(response.choices[0].message.content)
        print(f"\n📈 Tokens: {response.usage.total_tokens}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("\n" + "-"*80)

📊 Comparing model responses for:
'Given streamflow of 2500 CFS and rising 200 CFS/hour, should we issue a flood alert?'


🤖 Model: llama-3.3-nemotron-super-49b-v1.5

<think>
Okay, let's tackle this question. The user is asking if they should issue a flood alert given a streamflow of 2500 CFS that's rising at 200 CFS per hour. Hmm, first, I need to recall what factors are involved in determining a flood alert. 

I know that flood alerts are typically based on specific thresholds set by agencies like the National Weather Service (NWS) in the US. These thresholds vary depending on the location and the river's characteristics. So, the first thing I should consider is what the normal or safe streamflow levels are for this particular stream or river. Without knowing the specific location, it's hard to say for sure, but maybe there are general guidelines.

Streamflow of 2500 CFS—CFS stands for cubic feet per second, which is a measure of the volume of water flowing per second. The rising rate

### LLM-as-Judge Evaluation

The system includes an automatic evaluation feature using **cross-provider LLM-as-Judge**:
- When NVIDIA models generate responses, h2oGPTe judges them
- When h2oGPTe generates responses, NVIDIA models judge them
- This provides unbiased evaluation of response quality

Let's evaluate the NVIDIA model responses using our evaluation API:

In [9]:
import requests
import json

# Let's evaluate one of the model responses from above
question = "Given streamflow of 2500 CFS and rising 200 CFS/hour, should we issue a flood alert?"

# Response from meta/llama-3.1-70b-instruct (from cell 12)
response_text = """Streamflow is already high (2500 CFS) and rising rapidly (200 CFS/hour). I recommend issuing a flood alert immediately. The rapid increase in streamflow indicates a high risk of flooding, and prompt action is necessary to protect people and property."""

print("🔍 Evaluating NVIDIA model response using LLM-as-Judge...\n")

# Call the evaluation API
eval_payload = {
    "question": question,
    "response": response_text,
    "model": "meta/llama-3.1-70b-instruct",
    "agent_used": False,
    "response_provider": "nvidia"  # This will trigger h2oGPTe as the judge
}

headers = {"Authorization": "Bearer local-token"}
response = requests.post(
    f"{api_server_base_url}/api/evaluation/evaluate",
    json=eval_payload,
    headers=headers
)

if response.status_code == 200:
    eval_result = response.json()
    
    print("✅ Evaluation Complete!\n")
    print("="*80)
    print(f"📊 Evaluation Metrics:\n")
    
    metrics = eval_result.get('metrics', {})
    print(f"   🎯 Overall Score:    {metrics.get('overall', 0):.1f}/10")
    print(f"   🤝 Helpfulness:      {metrics.get('helpfulness', 0):.1f}/10")
    print(f"   ✅ Accuracy:         {metrics.get('accuracy', 0):.1f}/10")
    print(f"   🎯 Relevance:        {metrics.get('relevance', 0):.1f}/10")
    print(f"   📝 Coherence:        {metrics.get('coherence', 0):.1f}/10")
    print(f"   🛡️  Safety:           {metrics.get('safety', 0):.1f}/10")
    print(f"   💪 Confidence:       {metrics.get('confidence', 0):.1%}")
    
    print(f"\n💭 Judge's Reasoning:")
    print(f"   {eval_result.get('reasoning', 'N/A')}")
    
    print(f"\n⏱️  Evaluation Duration: {eval_result.get('duration_ms', 0)}ms")
    print(f"🆔 Evaluation ID: {eval_result.get('evaluation_id', 'N/A')}")
    print("="*80)
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

🔍 Evaluating NVIDIA model response using LLM-as-Judge...

✅ Evaluation Complete!

📊 Evaluation Metrics:

   🎯 Overall Score:    6.0/10
   🤝 Helpfulness:      6.0/10
   ✅ Accuracy:         4.0/10
   🎯 Relevance:        8.0/10
   📝 Coherence:        8.0/10
   🛡️  Safety:           7.0/10
   💪 Confidence:       85.0%

💭 Judge's Reasoning:
   The response is well-structured and directly addresses the question with appropriate urgency. However, it lacks critical context - flood thresholds vary dramatically by location, watershed characteristics, and baseline conditions. 2500 CFS could be normal flow for a large river or extreme flooding for a small stream. The response assumes high risk without considering flood stage levels, historical data, or local thresholds. While the precautionary approach is generally safe, making definitive recommendations without proper context could lead to unnecessary evacuations or alert fatigue. The response would benefit from requesting additional watershed-sp

---

# Section 3: h2oGPTe Agent (A2A) Integration

## 🧠 h2oGPTe - Enterprise AI with Agent Mode

h2oGPTe provides advanced AutoML capabilities through its agent mode, enabling:

- **Driverless AI Integration**: Automated machine learning with minimal code
- **Agent-to-Agent (A2A)**: AI agents that can invoke other AI agents
- **Feature Engineering**: Automatic feature creation for time-series data
- **Model Interpretability**: Explainable AI for emergency response decisions

### Setting up h2oGPTe Client

**Note**: This section requires h2oGPTe credentials. If you don't have access, you can skip to the next section.

Get your credentials at: [H2O.ai Enterprise](https://h2o.ai/platform/enterprise-h2ogpte/)

In [10]:
# Check if h2oGPTe credentials are available
h2ogpte_url = os.getenv('APP_H2OGPTE_URL', '')
h2ogpte_key = os.getenv('APP_H2OGPTE_API_KEY', '')

if h2ogpte_url and h2ogpte_key:
    print("✅ h2oGPTe credentials found")
    print(f"🔗 URL: {h2ogpte_url}")
    H2OGPTE_AVAILABLE = True
else:
    print("⚠️  h2oGPTe credentials not set")
    print("This section will be skipped. To enable:")
    print("  export APP_H2OGPTE_URL='<your-url>'")
    print("  export APP_H2OGPTE_API_KEY='<your-key>'")
    H2OGPTE_AVAILABLE = False

✅ h2oGPTe credentials found
🔗 URL: https://h2ogpte.cloud-dev.h2o.dev


### h2oGPTe for Flood Prediction ML

Let's use h2oGPTe's agent mode to get guidance on training a flood prediction model:

In [11]:
if H2OGPTE_AVAILABLE:
    import requests
    import json
    
    headers = {"Authorization": "Bearer local-token"}
    # Using FastAPI streaming endpoint for h2oGPTe
    url = f"{api_server_base_url}/api/ai/chat/enhanced/stream"
    
    payload = {
        "message": """I have flood prediction data with these features:
        - streamflow_cfs: Current river flow rate
        - rainfall_24h: Rainfall in last 24 hours
        - river_stage_ft: Water level
        - soil_moisture: Ground saturation
        - elevation_ft: Location elevation
        
        How should I approach building an ML model to predict flood risk in the next 24 hours?
        What feature engineering would you recommend?""",
        "provider": "h2ogpte",
        "use_agent": True,
        "max_tokens": 8192*10,
    }
    
    print("🧠 Consulting h2oGPTe agent for AutoML guidance...\n")
    
    response = requests.post(url, json=payload, headers=headers, stream=True)
    
    if response.status_code == 200:
        last_content = ""
        # Stream the response
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8')
                if line_str.startswith('data: '):
                    data_str = line_str[6:]  # Remove 'data: ' prefix
                    try:
                        data = json.loads(data_str)
                        
                        # First message contains provider info
                        if 'provider' in data:
                            print(f"📡 Provider: {data.get('provider')}")
                            print(f"🤖 Model: {data.get('model')}\n")
                            print("📝 Response:\n")
                        
                        # h2oGPTe sends incremental chunks with full content
                        elif 'chunk' in data and not data.get('done', False):
                            new_content = data['chunk']
                            # Only print the new portion
                            if new_content.startswith(last_content):
                                new_part = new_content[len(last_content):]
                                print(new_part, end='', flush=True)
                                last_content = new_content
                        
                        # Check for completion
                        elif data.get('done', False):
                            break
                            
                    except json.JSONDecodeError:
                        pass
        
        print("\n\n✅ Streaming complete!")
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
else:
    print("⏭️  Skipping h2oGPTe demo (credentials not available)")

🧠 Consulting h2oGPTe agent for AutoML guidance...

📡 Provider: h2ogpte
🤖 Model: claude-sonnet-4-20250514

📝 Response:

#### Agentic Analysis  
#### Starting Agent  
I'll help you build a comprehensive ML model for flood risk prediction. Let me start by analyzing your data structure and then provide recommendations for feature engineering and model building approaches.

```python
# filename: flood_prediction_analysis.py
# execution: true

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.se

---

# Section 4: Multi-Agent System with FastMCP

## 🤝 FastMCP - Model Context Protocol Server

Our flood prediction system uses FastMCP to expose 20+ specialized tools across 5 intelligent agents:

### The 5 Agents

1. **Data Collector Agent** 📊
   - Collects USGS water data
   - Retrieves NOAA flood forecasts
   - Gathers weather information
   - Monitors data quality

2. **Risk Analyzer Agent** ⚠️
   - Calculates flood risk scores
   - Analyzes trends and patterns
   - Identifies high-risk areas

3. **Emergency Responder Agent** 🚨
   - Assesses emergency readiness
   - Activates alerts
   - Coordinates evacuations

4. **AI Predictor Agent** 🔮
   - Generates flood forecasts
   - Predicts critical conditions
   - Analyzes prediction accuracy

5. **H2OGPTE ML Agent** 🧠
   - Trains ML models
   - Optimizes features
   - Analyzes model performance

### Let's explore the available tools:

In [12]:
# Get list of all agents and their capabilities
response = requests.get(f"{api_server_base_url}/api/agents")

if response.status_code == 200:
    data = response.json()
    
    print("🤖 Available Agents and Their Status\n")
    print("="*80)
    
    # The API returns a nested structure with agents dictionary
    agents_dict = data.get('agents', {})
    
    for agent_key, agent_data in agents_dict.items():
        status = "🟢" if agent_data.get('is_running') else "🔴"
        print(f"\n{status} {agent_data.get('name', agent_key)}")
        print(f"   Description: {agent_data.get('description', 'N/A')}")
        print(f"   Status: {'Running' if agent_data.get('is_running') else 'Stopped'}")
        if agent_data.get('last_check'):
            print(f"   Last Check: {agent_data.get('last_check')}")
        if agent_data.get('check_interval'):
            print(f"   Check Interval: {agent_data.get('check_interval')} seconds")
        if agent_data.get('insights_count'):
            print(f"   Insights: {agent_data.get('insights_count')}")
        if agent_data.get('active_alerts_count'):
            print(f"   Active Alerts: {agent_data.get('active_alerts_count')}")
    
    print("\n" + "="*80)
else:
    print(f"❌ Error fetching agents: {response.status_code}")

🤖 Available Agents and Their Status


🟢 Data Collector
   Description: Continuously pulls real-time flood data from USGS, NOAA, and weather APIs
   Status: Running
   Last Check: 2025-10-16T09:39:30.741364+00:00
   Check Interval: 300 seconds
   Insights: 4
   Active Alerts: 6

🟢 Risk Analyzer
   Description: AI-powered analysis of flood risk conditions and trend detection
   Status: Running
   Last Check: 2025-10-16T09:37:26.977038+00:00
   Check Interval: 600 seconds
   Insights: 4
   Active Alerts: 10

🟢 Emergency Responder
   Description: Coordinates emergency response activities and manages critical alerts
   Status: Running
   Last Check: 2025-10-16T09:41:27.049336+00:00
   Check Interval: 180 seconds
   Insights: 5
   Active Alerts: 33

🟢 AI Predictor
   Description: Advanced AI forecasting and predictive analysis for flood conditions
   Status: Running
   Last Check: 2025-10-16T09:32:27.009585+00:00
   Check Interval: 900 seconds
   Insights: 4
   Active Alerts: 1



### View Agent Insights

Agents continuously monitor flood conditions and generate insights:

In [13]:
import time
# Refresh USGS data before getting insights
print("🔄 Refreshing USGS data first...\n")

# Note: Using local-token for development mode (server.py allows this when OIDC is disabled)
headers = {"Authorization": "Bearer local-token"}

refresh_response = requests.post(
    f"{api_server_base_url}/api/dashboard/refresh-usgs-data",
    headers=headers
)

if refresh_response.status_code == 200:
    refresh_result = refresh_response.json()
    print(f"✅ {refresh_result.get('message', 'Data refresh initiated')}")
    
    # Wait a moment for background job to start
    print("⏳ Waiting for data refresh to process...\n")
    time.sleep(3)
else:
    print(f"⚠️  Data refresh returned status {refresh_response.status_code}")
    print(f"   Proceeding with existing data...\n")

# Get insights from all agents
response = requests.get(f"{api_server_base_url}/api/agents/insights")

if response.status_code == 200:
    data = response.json()
    
    print("💡 Agent Insights\n")
    print("="*80)
    
    # The API returns insights grouped by agent
    insights_by_agent = data.get('insights', {})
    
    count = 0
    for agent_name, agent_insights in insights_by_agent.items():
        print(f"\n🤖 {agent_name.replace('_', ' ').title()}")
        print("-" * 60)
        
        for insight in agent_insights:
            title = insight.get('title', 'N/A')
            value = insight.get('value', 'N/A')
            change = insight.get('change')
            urgency = insight.get('urgency', 'normal')
            timestamp = insight.get('timestamp', '')
            
            urgency_icon = {
                'critical': '🔴',
                'high': '🟡',
                'normal': '🔵',
                'low': '🟢'
            }.get(urgency, '⚪')
            
            print(f"\n{urgency_icon} {title}: {value}")
            if change:
                print(f"   Change: {change}")
            
            count += 1
            if count >= 15:  # Limit total insights shown
                break
        
        if count >= 15:
            break
    
    print("\n" + "="*80)
    print(f"Generated at: {data.get('generated_at', 'N/A')}")
else:
    print(f"❌ Error fetching insights: {response.status_code}")

🔄 Refreshing USGS data first...

✅ USGS data refresh started
⏳ Waiting for data refresh to process...

💡 Agent Insights


🤖 Data Collector
------------------------------------------------------------

🟡 🔄 Data Freshness: 0% current

🔵 🌐 API Connectivity: 5/5 active

🔵 📊 Data Quality: 6.0/10
   Change: +0.0

🔵 ⚡ Update Frequency: 12 updates/hour

🤖 Risk Analyzer
------------------------------------------------------------

🔵 🎯 Overall Risk Level: LOW (1.8/10)
   Change: +0.0

🔵 🚨 Critical Watersheds: 0 areas

🔵 📈 Risk Trend Analysis: Stable
   Change: 0.0% per hour

🔵 🧠 AI Confidence: 40%
   Change: Low

🤖 Emergency Responder
------------------------------------------------------------

🔵 🚨 Active Incidents: 0 ongoing

🔵 🚁 Response Readiness: GOOD (85%)
   Change: 8 teams ready

🔵 🏃 Evacuation Status: No active evacuations
   Change: 0 zones active

🔵 📡 Communication Systems: 100% operational
   Change: 6 channels active

🔵 📢 Alert Distribution: 0 alerts sent
   Change: 0 today

🤖 Predi

### Example 1: Risk Analyzer Agent

Calculates comprehensive flood risk scores:

In [14]:
# Run Risk Analyzer Agent via NAT
import json
payload = {
    "agent": "risk_analyzer",
    "message": """Analyze current flood risk for the Texas:
    1. Calculate detailed risk scores for all factors
    2. Identify the highest risk components
    3. Provide trend analysis
    4. Give recommendations for monitoring
    
    Be specific about the risk levels and factors."""
}

print("Running Risk Analyzer Agent...\n")

headers = {"Authorization": "Bearer local-token"}

response = requests.post(
    f"{api_server_base_url}/api/nat/chat/stream",
    json=payload,
    headers=headers,
    stream=True
)

if response.status_code == 200:
    final_output = None
    
    for line in response.iter_lines():
        if line:
            line_str = line.decode('utf-8')
            if line_str.startswith('data: '):
                data_str = line_str[6:]  # Remove 'data: ' prefix
                try:
                    data = json.loads(data_str)
                    
                    # Handle different event types
                    event_type = data.get('type')
                    
                    if event_type == 'start':
                        print(f"🚀 Starting {data.get('agent_type')} agent...")
                        print()
                    
                    elif event_type == 'log':
                        log_entry = data.get('log', {})
                        level = log_entry.get('level', 'INFO')
                        message = log_entry.get('message', '')
                        
                        # Show important logs
                        if level in ['WARNING', 'ERROR']:
                            print(f"[{level}] {message}")
                        elif 'Agent' in message or 'Final Answer' in message or 'Tool' in message:
                            print(f"💬 {message}")
                    
                    elif event_type == 'result':
                        final_output = data.get('output')
                        print("\n" + "="*80)
                        print("✅ Data Collection Complete!\n")
                    
                    elif event_type == 'error':
                        print(f"\n❌ Error: {data.get('error')}")
                    
                    elif event_type == 'done':
                        break
                        
                except json.JSONDecodeError:
                    pass
    
    # Display final output
    if final_output:
        print(final_output)
        print("="*80)
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

Running Risk Analyzer Agent...

🚀 Starting risk_analyzer agent...

[WARNING] Using provided input_schema for multi-argument function
[WARNING] Using provided input_schema for multi-argument function
[WARNING] Using provided input_schema for multi-argument function
💬 
------------------------------
[AGENT]
Agent input: Analyze current flood risk for the Texas:
    1. Calculate detailed risk scores for all factors
    2. Identify the highest risk components
    3. Provide trend analysis
    4. Give recommendations for monitoring

    Be specific about the risk levels and factors.
Agent's thoughts: 
k>

Action: comprehensive_flood_analysis_tool
Action Input: {"location": "Texas"}


------------------------------
💬 
------------------------------
[AGENT]
Calling tools: comprehensive_flood_analysis_tool
Tool's input: {'location': 'Texas'}
Tool's response: 
{"status":"success","location":"Texas","analysis_timestamp":"2025-10-16T09:44:19.281009+00:00","data_collection":{"agent":"Data Collecto

### Example 2: H2OGPTE ML Agent

AutoML agent for model training and optimization:

In [ ]:
# Run H2OGPTE ML Agent via NAT
payload = {
    "agent": "h2ogpte_agent",
    "message": """Help me design an ML pipeline for flood prediction:
    1. What features should I engineer from raw sensor data?
    2. What model types work best for flood forecasting?
    3. How should I handle imbalanced flood event data?
    4. What validation strategy is appropriate for time-series?
    
    Provide actionable AutoML recommendations."""
}
headers = {"Authorization": "Bearer local-token"}

print("🧠 Running H2OGPTE ML Agent...\n")

response = requests.post(f"{api_server_base_url}/api/nat/chat", json=payload, headers=headers)

if response.status_code == 200:
    result = response.json()
    print("✅ ML Recommendations Complete!\n")
    print("="*80)
    print(result.get('response', result))
    print("="*80)
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

---

# Section 6: Real-World Data Integration

## 🌐 Live Data from Government APIs

Our system integrates with real-time data sources:

### Data Sources

1. **USGS Water Services**
   - Real-time streamflow (CFS)
   - Gage height (feet)
   - 12 monitoring stations in Texas
   - Updated every 15 minutes

2. **NOAA Weather Service**
   - Flood warnings and watches
   - Weather alerts
   - Forecast data

3. **Open-Meteo**
   - Weather forecasts
   - Flood API predictions
   - Historical data

### Let's view live watershed data:

##### Refresh USGS Data Manually

Trigger a fresh data collection from USGS:

In [39]:
print("🔄 Triggering USGS data refresh...\n")
header = {"Authorization": "Bearer local-token"}

response = requests.post(f"{api_server_base_url}/api/dashboard/refresh-usgs-data", headers=header    )

if response.status_code == 200:
    result = response.json()
    time.sleep(5)
    
else:
    print(f"❌ Error: {response.status_code}")
    
import pandas as pd

# Get current watershed data
response = requests.get(f"{api_server_base_url}/api/watersheds")

if response.status_code == 200:
    watersheds = response.json()
    
    # Convert to DataFrame for nice display
    df = pd.DataFrame(watersheds)
    
    # Select key columns
    display_cols = ['name', 'current_streamflow_cfs', 'risk_score', 
                    'trend_rate_cfs_per_hour', 'last_updated']
    
    available_cols = [col for col in display_cols if col in df.columns]
    
    print("🌊 Live Watershed Data\n")
    print("="*100)
    print(df[available_cols].to_string(index=False))
    print("="*100)
    print(f"\n📊 Total Watersheds Monitored: {len(watersheds)}")
    
    # Calculate statistics
    if 'risk_score' in df.columns:
        high_risk = len(df[df['risk_score'] > 7.0])
        medium_risk = len(df[(df['risk_score'] >= 4.0) & (df['risk_score'] <= 7.0)])
        low_risk = len(df[df['risk_score'] < 4.0])
        
        print(f"\n⚠️  Risk Distribution:")
        print(f"   🔴 High Risk (>7.0): {high_risk}")
        print(f"   🟡 Medium Risk (4.0-7.0): {medium_risk}")
        print(f"   🟢 Low Risk (<4.0): {low_risk}")
else:
    print(f"❌ Error fetching watersheds: {response.status_code}")

🔄 Triggering USGS data refresh...



🌊 Live Watershed Data

                                                name  current_streamflow_cfs  risk_score  trend_rate_cfs_per_hour        last_updated
         Neches River at Evadale, TX (USGS 08041000)                 1000.00         4.0                      0.0 2025-10-04 20:00:07
        Red River at Arthur City, TX (USGS 07335500)                 1020.00         4.0                      0.0 2025-10-04 20:00:06
      Brazos River near Rosharon, TX (USGS 08116650)                  934.00         3.9                      0.0 2025-10-04 20:00:07
         Trinity River at Dallas, TX (USGS 08057000)                  330.00         1.6                      0.0 2025-10-04 20:00:07
        Colorado River at Austin, TX (USGS 08158000)                  249.00         1.2                      0.0 2025-10-04 20:00:07
    Sabine River near Gladewater, TX (USGS 08020000)                  120.00         0.6                      0.0 2025-10-04 20:00:07
      Guadalupe River at Comfort, TX (U

---

# 🎓 Summary & Next Steps

## What We've Learned

In this notebook, you've learned how to:

✅ **Set up a multi-agent AI system** for disaster response  
✅ **Integrate h2oGPTe** for AutoML and model training  
✅ **Use NVIDIA NIM** for high-performance inference  
✅ **Build NAT agent workflows** with React patterns  
✅ **Implement FastMCP servers** with custom tools  
✅ **Integrate real-time data** from government APIs  
✅ **Coordinate multiple AI agents** for complex tasks  
✅ **Evaluate responses** using LLM-as-Judge  

## Architecture Highlights

- **5 Specialized Agents**: Data Collector, Risk Analyzer, Emergency Responder, Predictor, H2OGPTE ML
- **20+ MCP Tools**: Via FastMCP server on port 8001
- **NVIDIA NIM Models**: Nemotron Super 49B, Llama 3.1 variants
- **h2oGPTe A2A**: Agent-to-agent AutoML capabilities
- **Real-time Data**: USGS, NOAA, Weather APIs

## Resources

- **NVIDIA NIM**: [build.nvidia.com](https://build.nvidia.com)
- **h2oGPTe**: [h2o.ai/platform/enterprise-h2ogpte](https://h2o.ai/platform/enterprise-h2ogpte/)
- **NVIDIA NAT**: [docs.nvidia.com/nat](https://docs.nvidia.com/nat)
- **FastMCP**: [github.com/jlowin/fastmcp](https://github.com/jlowin/fastmcp)
- **USGS Water Data**: [waterdata.usgs.gov](https://waterdata.usgs.gov)
- **NOAA Weather**: [weather.gov](https://weather.gov)

## Next Steps

1. **Customize Agents**: Modify agent configs for your specific use case
2. **Add Data Sources**: Integrate additional APIs and sensors
3. **Train ML Models**: Use h2oGPTe to train production models
4. **Deploy to Production**: Use Docker/Kubernetes deployment
5. **Monitor Performance**: Add logging and metrics

## Contributing

This is an open-source AI for Good project. Contributions welcome!

---

### 🌊 Thank you for exploring the Flood Prediction Blueprint!

**Built with ❤️ using h2oGPTe and NVIDIA NIM**

For questions and support, please open an issue in the repository.